In [2]:
import pandas as pd
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import sqlite3


In [90]:
tmdb = pd.read_csv('zippedData/tmdb.movies.csv.gz', index_col = 0)
tmdb['movie'] = tmdb['title']
tmdb = tmdb.drop(['title'], axis = 1)
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   vote_average       26517 non-null  float64
 7   vote_count         26517 non-null  int64  
 8   movie              26517 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


In [91]:
tn = pd.read_csv('zippedData/tn.movie_budgets.csv.gz', index_col = 0)
tn.info()
tn.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   object
 3   domestic_gross     5782 non-null   object
 4   worldwide_gross    5782 non-null   object
dtypes: object(5)
memory usage: 271.0+ KB


,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [12]:
## link for https://developers.themoviedb.org/3/getting-started/popularity

In [89]:
bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
bom['movie'] = bom['title']
bom = bom.drop(['title'], axis = 1)

In [127]:
rt_movie = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter = '\t', encoding = 'unicode_escape')
rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter = '\t', encoding = 'unicode_escape',)

In [131]:
rt_reviews_con = rt_reviews.drop(['review', 'critic', 'publisher', 'date'], axis=1)
rtr = rt_reviews_con

In [132]:
rtr['rating'] = rtr.loc[rtr['rating'].str.contains('/5') == True]['rating'].apply(lambda x: int(x[0]) / int(x[-1])  if len(x) == 3 else  float(x[0:2]) / int(x[-1])    )
rtr_nona = rtr.dropna(subset = ['rating'])
rtr_mean = rtr_nona.groupby(['id'])[['rating']].agg(['mean'])*100
rtr = rtr.merge(rtr_mean, on=['id'], how = 'left')

C:\Users\paperspace\anaconda3\envs\learn-env\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [102]:
tn_bom = tn.merge(bom, on=['movie'], how='left')
tn_bom = tn_bom[['movie', 'year', 'release_date', 'studio', 'production_budget', 'domestic_gross_x', 'domestic_gross_y', 'worldwide_gross', 'foreign_gross']]

In [135]:
tmdb_tn_bom = tmdb.merge(tn_bom, on=['movie'], how='left')
tmdb_tn_bom = tmdb_tn_bom.drop(['id', 'original_language', 'original_title', 'release_date_y', 'domestic_gross_y'], axis = 1)
tmdb_tn_bom = tmdb_tn_bom[['movie', 'release_date_x', 'year', 'studio', 'genre_ids', 'production_budget', 'domestic_gross_x','worldwide_gross', 'foreign_gross', 'popularity','vote_average','vote_count']]
tmdb_tn_bom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26606 entries, 0 to 26605
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie              26606 non-null  object 
 1   release_date_x     26606 non-null  object 
 2   year               1395 non-null   float64
 3   studio             1394 non-null   object 
 4   genre_ids          26606 non-null  object 
 5   production_budget  2385 non-null   object 
 6   domestic_gross_x   2385 non-null   object 
 7   worldwide_gross    2385 non-null   object 
 8   foreign_gross      1200 non-null   object 
 9   popularity         26606 non-null  float64
 10  vote_average       26606 non-null  float64
 11  vote_count         26606 non-null  int64  
dtypes: float64(3), int64(1), object(8)
memory usage: 2.6+ MB


In [133]:
rt_movie_con = rt_movie.drop(['synopsis', 'dvd_date', 'rating'], axis=1)
rt_movie_con['Box Office'] = rt_movie_con['currency'] + rt_movie_con['box_office']
rt_movie_con = rt_movie_con.drop(['currency', 'box_office'], axis = 1)
rtm = rt_movie_con
rtm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   genre         1552 non-null   object
 2   director      1361 non-null   object
 3   writer        1111 non-null   object
 4   theater_date  1201 non-null   object
 5   runtime       1530 non-null   object
 6   studio        494 non-null    object
 7   Box Office    340 non-null    object
dtypes: int64(1), object(7)
memory usage: 97.6+ KB


In [134]:
rtmr = rtm.merge(rtr, on=['id'], how='left')
rtmr.info()
rtmr.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54857 entries, 0 to 54856
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              54857 non-null  int64  
 1   genre           54764 non-null  object 
 2   director        49339 non-null  object 
 3   writer          45426 non-null  object 
 4   theater_date    53411 non-null  object 
 5   runtime         54001 non-null  object 
 6   studio          40204 non-null  object 
 7   Box Office      33351 non-null  object 
 8   rating          17043 non-null  float64
 9   fresh           54432 non-null  object 
 10  top_critic      54432 non-null  float64
 11  (rating, mean)  54293 non-null  float64
dtypes: float64(3), int64(1), object(8)
memory usage: 5.4+ MB


,id,genre,director,writer,theater_date,runtime,studio,Box Office,rating,fresh,top_critic,"(rating, mean)"
0,1,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971",104 minutes,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012",108 minutes,Entertainment One,"$600,000",0.6,fresh,0.0,59.6
2,3,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012",108 minutes,Entertainment One,"$600,000",NaN,rotten,0.0,59.6
3,3,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012",108 minutes,Entertainment One,"$600,000",NaN,fresh,0.0,59.6
4,3,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012",108 minutes,Entertainment One,"$600,000",NaN,fresh,0.0,59.6


**Certified Fresh status is a special distinction awarded to the best-reviewed movies and TV shows. In order to qualify, movies or TV shows must meet the following requirements:

A consistent Tomatometer score of 75% or higher.
At least five reviews from Top Critics.
Films in wide release must have a minimum of 80 reviews. This also applies for films going from limited to wide release.
Films in limited release must have a minimum of 40 reviews.
Only individual seasons of a TV show are eligible, and each must have a minimum of 20 reviews.

In [92]:
# rt_movie_con['box_office'].isna().sum()
# rt_movie_con['currency'].isna().sum()



In [3]:
rtcurrent = pd.read_csv('zippedData/rotten_tomatoes_movies.csv')
rtcurrent.head()


,movie_title,movie_info,content_rating,genres,directors,authors,actors,original_release_date,production_company,tomatometer_status
0,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2/12/2010,20th Century Fox,Rotten
1,Please Give,Kate (Catherine Keener) and her husband Alex (...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",4/30/2010,Sony Pictures Classics,Certified-Fresh
2,10,"A successful, middle-aged Hollywood songwriter...",R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",10/5/1979,Waner Bros.,Fresh
3,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",4/13/1957,Criterion Collection,Certified-Fresh
4,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1/1/1954,Disney,Fresh


In [136]:
# rtcurrent_con = rtcurrent['movie_title']['genres']['directors']['authors']['actors']['original_release_date']['production_company']
# rtcurrent_con.head()

KeyError: 'genres'

In [40]:
conn = sqlite3.connect('zippedData/im.db/im.db')

In [23]:
cursor = con.cursor()

In [37]:
pd.read_sql = ("""
SELECT name
FROM sqlite_master
where type = 'table'
""", con)


In [ ]:
# SELECT name
# FROM sqlite_master
# where type = 'table'
# """, con


In [44]:
pd.read_sql("""
SELECT *
FROM movie_basics
JOIN directors
    USING (movie_id)
JOIN movie_ratings
    USING (person_id)
JOIN persons
    USING (person_id)
JOIN principals
    USING (person_id)
    """,conn) 

TypeError: 'tuple' object is not callable